In [ ]:
import numpy as np
import xarray as xr

import os
from dolfin import *


from mmdisk.fea.fenics.utils import get_loadfunc, get_time_list_alt
from mmdisk.fea.common import create_time_points
from mmdisk.fea.fenics.mesh import mesh_and_properties, mesh2d_and_properties
from mmdisk.fea.fenics.incremental import incremental_cyclic_fea
from mmdisk.fea.common import default_cycle

import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'mmdisk'

In [ ]:
disks2d = xr.open_zarr("one_disk_with_full_fgm_comp_prop_1000_Cre_calib_rev_fix_chunk_1_for_fea.zarr")
mesh, props = mesh_and_properties(disks2d.isel(disks=0).load())
# mesh, props = mesh_and_properties(picked_disk.load())
omega = 1000
T_load = 200
t_heatdwell = 20.0
t_cooldwell = 1500.  # cooling dwell time
skip_cold = False  # skip the cold dwell for the thermal problem

l_cycle = default_cycle.copy()
l_cycle[1] = t_heatdwell
l_cycle[3] = t_cooldwell

time_division =  [0.25, 0.25, 0.25, 0.25]

time_steps = l_cycle.copy() * time_division

load = get_loadfunc(T_load, l_cycle)

n_cycles = 10

# t_fine = l_cycle[:3].sum() + l_cycle[2]
period, _, t_output = create_time_points(n_cycles, l_cycle, output_division=2.0)
t_list = get_time_list_alt(l_cycle, time_steps, n_cycles, skip_cold=skip_cold)

In [ ]:
mesh = mesh
properties = props
load = T_load
omega = omega
t_list = t_list
period = period
t_output = t_output

In [ ]:
# material properties
rho, cp, kappa, E, sig0, nu, alpha_V = properties
lmbda = E * nu / (1 + nu) / (1 - 2 * nu)
mu = E / 2.0 / (1 + nu)
Et = E / 1e5  # tangent modulus - any better method for perfect plasticity?
H = E * Et / (E - Et)  # hardening modulus
